In [ ]:
# INSTALLATION / MISE À JOUR COMFYUI AUTOMATIQUE

import os
import subprocess

COMFY_PATH = "/workspace/ComfyUI"

print("🚀 Vérification de l'installation de ComfyUI...")

if os.path.exists(COMFY_PATH):
    print("📂 ComfyUI détecté. Mise à jour en cours...")
    # On force la mise à jour vers la dernière version
    try:
        # git fetch récupère les infos, git reset --hard force l'état local à correspondre exactement au serveur
        subprocess.run(f"cd {COMFY_PATH} && git fetch origin && git reset --hard origin/master", shell=True, check=True)
        print("✅ Mise à jour réussie (Dernier commit master).")
    except Exception as e:
        print(f"⚠️ Erreur lors de la mise à jour : {e}")
        print("   -> Tentative de git pull classique...")
        subprocess.run(f"cd {COMFY_PATH} && git pull", shell=True)
else:
    print("📥 Première installation (Clonage)...")
    subprocess.run(f"git clone https://github.com/comfyanonymous/ComfyUI.git {COMFY_PATH}", shell=True)
    print("✅ Installation terminée.")


In [1]:
# === INSTALLATION CUSTOM NODES (Nodes + Modèles Vision + Dépendances + Fix NSFW) ===

import os
import subprocess
import sys

COMFY_DIR = "/workspace/ComfyUI"
CUSTOM_NODES_DIR = f"{COMFY_DIR}/custom_nodes"

print("🚀 DÉMARRAGE DE L'INSTALLATION FLUX & TOOLS...")

# --- 0. INSTALLATION SYSTÈME (ARIA2C) ---
# Ajouté pour corriger l'erreur FileNotFoundError
print("🛠️ Vérification/Installation de Aria2c...")
try:
    # On tente d'installer aria2 via apt-get
    subprocess.run(["apt-get", "update"], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    subprocess.run(["apt-get", "install", "-y", "aria2"], check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    print("   ✅ Aria2c est prêt.")
except Exception as e:
    print(f"   ⚠️ Impossible d'installer Aria2c (Check permissions): {e}")
    print("   ⚠️ Le script tentera de continuer, mais les téléchargements rapides pourraient échouer.")


# --- 1. LISTE DES NODES (Pack Image Pure) ---
nodes_to_install = [
    "https://github.com/Comfy-Org/ComfyUI-Manager",             # Manager
    "https://github.com/rgthree/rgthree-comfy",                 # Rgthree
    "https://github.com/lldacing/ComfyUI_PuLID_Flux_ll",        # PuLID_Flux_ll
    "https://github.com/cubiq/ComfyUI_essentials",              # Essentials    
    "https://github.com/pythongosssss/ComfyUI-Custom-Scripts",  # Interface    
    "https://github.com/ltdrdata/ComfyUI-Impact-Pack",          # Impact Pack (Face Detailer)
    "https://github.com/ltdrdata/ComfyUI-Impact-Subpack",       # Impact Subpack
    "https://github.com/numz/ComfyUI-SeedVR2_VideoUpscaler",    # SeedVR2       
    "https://github.com/Gourieff/ComfyUI-ReActor",              # ReActor (Face Swap)   
    "https://github.com/jags111/efficiency-nodes-comfyui",      # Ensemble de Nodes
    "https://github.com/city96/ComfyUI-GGUF",                   # GGUF
    
]

# --- 2. INSTALLATION DES NODES ---
# Pré-installation InsightFace (Critique pour ReActor)
try:
    import insightface
except ImportError:
    print("📦 Installation de InsightFace & ONNX...")
    subprocess.run([sys.executable, "-m", "pip", "install", "insightface", "onnxruntime-gpu"], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

for repo_url in nodes_to_install:
    node_name = repo_url.split("/")[-1]
    node_path = os.path.join(CUSTOM_NODES_DIR, node_name)
    
    if not os.path.exists(node_path):
        print(f"⬇️ Clonage de : {node_name}...")
        try:
            subprocess.run(["git", "clone", repo_url, node_path], check=True, stdout=subprocess.DEVNULL)
            req_path = os.path.join(node_path, "requirements.txt")
            if os.path.exists(req_path):
                print(f"   📦 Dépendances pour {node_name}...")
                subprocess.run([sys.executable, "-m", "pip", "install", "-r", req_path], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
        except subprocess.CalledProcessError:
            print(f"   ❌ Erreur clone {node_name}")
    else:
        print(f"⏩ [SKIP] {node_name} ok.")

# --- 3. TÉLÉCHARGEMENT MODÈLES ULTRALYTICS (YOLO - Pour Impact Pack) ---
print("\n👁️ Vérification des modèles de détection (YOLO)...")
base_yolo = f"{COMFY_DIR}/models/ultralytics"
bbox_path = os.path.join(base_yolo, "bbox")
segm_path = os.path.join(base_yolo, "segm")
os.makedirs(bbox_path, exist_ok=True)
os.makedirs(segm_path, exist_ok=True)

yolo_models = [
    # BBOX (Carrés)
    ("face_yolov8m.pt", "https://huggingface.co/Bingsu/adetailer/resolve/main/face_yolov8m.pt", bbox_path),
    ("hand_yolov8s.pt", "https://huggingface.co/Bingsu/adetailer/resolve/main/hand_yolov8s.pt", bbox_path),
    # SEGM (Masques précis)
    ("person_yolov8m-seg.pt", "https://huggingface.co/Bingsu/adetailer/resolve/main/person_yolov8m-seg.pt", segm_path),
    ("yolov8n-seg.pt", "https://huggingface.co/Bingsu/adetailer/resolve/main/yolov8n-seg.pt", segm_path),
    ("face_yolov8m-seg_60.pt", "https://huggingface.co/bobmin/face_yolov8m-seg_60/resolve/main/face_yolov8m-seg_60.pt", segm_path)
]

for name, url, dest in yolo_models:
    if not os.path.exists(os.path.join(dest, name)):
        print(f"   ⬇️ YOLO Download : {name}")
        # L'appel à aria2c fonctionnera maintenant car installé à l'étape 0
        try:
            subprocess.run(["aria2c", "-x", "16", "-s", "16", "-k", "1M", "-d", dest, "-o", name, url], check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
        except FileNotFoundError:
            # Fallback de secours si l'installation d'aria2 a échoué (utilise wget)
            print(f"   ⚠️ Aria2c manquant, tentative avec wget pour {name}...")
            subprocess.run(["wget", "-O", os.path.join(dest, name), url], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

# --- 4. PATCH REACTOR NSFW (AUTOMATIQUE) ---
print("\n🔓 Application du patch ReActor (NSFW Bypass)...")
reactor_path = f"{CUSTOM_NODES_DIR}/ComfyUI-ReActor/scripts/reactor_sfw.py"
bypass_code = """
from PIL import Image
import logging
import os
from scripts.reactor_logger import logger
MODEL_EXISTS = True
def ensure_nsfw_model(nsfwdet_model_path): return True
def nsfw_image(img_data, model_path: str): return False
"""

if os.path.exists(os.path.dirname(reactor_path)):
    try:
        with open(reactor_path, "w") as f: f.write(bypass_code)
        print("   ✅ Patch appliqué sur 'reactor_sfw.py'.")
    except Exception as e:
        print(f"   ❌ Erreur patch : {e}")
else:
    print("   ⚠️ ReActor non trouvé, patch ignoré.")

print("\n🎉 ENVIRONNEMENT FLUX COMPLET ET PRÊT !")
print(">>> Relancez la cellule de lancement (ComfyUI) pour charger tout ça.")

🚀 DÉMARRAGE DE L'INSTALLATION FLUX & TOOLS...
📦 Installation de InsightFace & ONNX...
⏩ [SKIP] ComfyUI-Manager ok.
⏩ [SKIP] rgthree-comfy ok.
⏩ [SKIP] ComfyUI_PuLID_Flux_ll ok.
⏩ [SKIP] ComfyUI_essentials ok.
⏩ [SKIP] ComfyUI-Custom-Scripts ok.
⏩ [SKIP] ComfyUI-Impact-Pack ok.
⏩ [SKIP] ComfyUI-Impact-Subpack ok.
⏩ [SKIP] ComfyUI-SeedVR2_VideoUpscaler ok.
⏩ [SKIP] ComfyUI-ReActor ok.
⏩ [SKIP] efficiency-nodes-comfyui ok.
⏩ [SKIP] ComfyUI-GGUF ok.

👁️ Vérification des modèles de détection (YOLO)...
   ⬇️ YOLO Download : yolov8n-seg.pt


FileNotFoundError: [Errno 2] No such file or directory: 'aria2c'

In [ ]:
# Z-IMAGE TURBO FILES DOWNLOAD

import os
import ipywidgets as widgets
from IPython.display import display, clear_output

# --- 1. CONFIGURATION SÉCURISÉE ---
CIVITAI_TOKEN = os.environ.get("CIVITAI_TOKEN", "")
HF_TOKEN = os.environ.get("HF_TOKEN", "")

# Installation silencieuse de aria2
os.system("apt-get update -y > /dev/null && apt-get install -y aria2 > /dev/null")

# --- 2. BASE DE DONNÉES (Z-Image Turbo) ---
db_models = {
    "🧬 Z-Image Turbo Core": [
        ("[11.7 GB] Z-Image Turbo BF16", "https://huggingface.co/Comfy-Org/z_image_turbo/resolve/main/split_files/diffusion_models/z_image_turbo_bf16.safetensors?download=true", "/workspace/ComfyUI/models/diffusion_models", "z_image_turbo_bf16.safetensors", "hf"),
        ("[320 MB] VAE (ae.safetensors)", "https://huggingface.co/Comfy-Org/z_image_turbo/resolve/main/split_files/vae/ae.safetensors?download=true", "/workspace/ComfyUI/models/vae", "ae.safetensors", "hf"),
        ("[7.5 GB] Qwen 3.4B (Text Enc)", "https://huggingface.co/Comfy-Org/z_image_turbo/resolve/main/split_files/text_encoders/qwen_3_4b.safetensors?download=true", "/workspace/ComfyUI/models/text_encoders", "qwen_3_4b.safetensors", "hf"),
    ],
    "🧪 LoRAs Personnels (ZIT Training)": [
        ("Grainscape Z-Image", "https://huggingface.co/Danrisi/GrainScape_UltraReal_ZImage/resolve/main/grainscape_zimage.safetensors", "/workspace/ComfyUI/models/loras", "grainscape_zimage.safetensors", "hf"),
        ("Tumblr Minimalism 2250", "https://huggingface.co/rafatribe/Personnal_Loras/resolve/main/tmblr0ne_minimalism_000002250.safetensors?download=true", "/workspace/ComfyUI/models/loras", "tmblr0ne_minimalism_000002250.safetensors", "hf"),
        ("Tumblr Minimalism 2750", "https://huggingface.co/rafatribe/Personnal_Loras/resolve/main/tmblr0ne_minimalism_000002750.safetensors?download=true", "/workspace/ComfyUI/models/loras", "tmblr0ne_minimalism_000002750.safetensors", "hf"),
        ("Hairfix 1500", "https://huggingface.co/rafatribe/Personnal_Loras/resolve/main/v3sna-ZIT-hairfix_000001500.safetensors?download=true", "/workspace/ComfyUI/models/loras", "v3sna-ZIT-hairfix_000001500.safetensors", "hf"),
        ("Hairfix 1750", "https://huggingface.co/rafatribe/Personnal_Loras/resolve/main/v3sna-ZIT-hairfix_000001750.safetensors?download=true", "/workspace/ComfyUI/models/loras", "v3sna-ZIT-hairfix_000001750.safetensors", "hf"),
        ("Hairfix 2000", "https://huggingface.co/rafatribe/Personnal_Loras/resolve/main/v3sna-ZIT-hairfix_000002000.safetensors?download=true", "/workspace/ComfyUI/models/loras", "v3sna-ZIT-hairfix_000002000.safetensors", "hf"),
        ("Hairfix 2250", "https://huggingface.co/rafatribe/Personnal_Loras/resolve/main/v3sna-ZIT-hairfix_000002250.safetensors?download=true", "/workspace/ComfyUI/models/loras", "v3sna-ZIT-hairfix_000002250.safetensors", "hf"),
        ("Vy3sna Woman 2250", "https://huggingface.co/rafatribe/Personnal_Loras/resolve/main/vy3snawoman-zit_000002250.safetensors?download=true", "/workspace/ComfyUI/models/loras", "vy3snawoman-zit_000002250.safetensors", "hf"),
        ("Vy3sna Woman 2500", "https://huggingface.co/rafatribe/Personnal_Loras/resolve/main/vy3snawoman-zit_000002500.safetensors?download=true", "/workspace/ComfyUI/models/loras", "vy3snawoman-zit_000002500.safetensors", "hf"),
        ("Vy3sna Woman 2750", "https://huggingface.co/rafatribe/Personnal_Loras/resolve/main/vy3snawoman-zit_000002750.safetensors?download=true", "/workspace/ComfyUI/models/loras", "vy3snawoman-zit_000002750.safetensors", "hf"),
        ("Vy3sna Woman 3000", "https://huggingface.co/rafatribe/Personnal_Loras/resolve/main/vy3snawoman-zit_000003000.safetensors?download=true", "/workspace/ComfyUI/models/loras", "vy3snawoman-zit_000003000.safetensors", "hf"),
        ("Vy3sna Woman 3250", "https://huggingface.co/rafatribe/Personnal_Loras/resolve/main/vy3snawoman-zit_000003250.safetensors?download=true", "/workspace/ComfyUI/models/loras", "vy3snawoman-zit_000003250.safetensors", "hf"),
        ("Vyesna ZIT v1 1750", "https://huggingface.co/rafatribe/Personnal_Loras/resolve/main/Vyesna-ZIT-lora_v1_000001750.safetensors?download=true", "/workspace/ComfyUI/models/loras", "Vyesna-ZIT-lora_v1_000001750.safetensors", "hf"),
        ("Vyesna ZIT v1 2250", "https://huggingface.co/rafatribe/Personnal_Loras/resolve/main/Vyesna-ZIT-lora_v1_000002250.safetensors?download=true", "/workspace/ComfyUI/models/loras", "Vyesna-ZIT-lora_v1_000002250.safetensors", "hf"),
        ("Vyesna ZIT v1 2500", "https://huggingface.co/rafatribe/Personnal_Loras/resolve/main/Vyesna-ZIT-lora_v1_000002500.safetensors?download=true", "/workspace/ComfyUI/models/loras", "Vyesna-ZIT-lora_v1_000002500.safetensors", "hf"),
        ("Vyesna ZIT v1 2750", "https://huggingface.co/rafatribe/Personnal_Loras/resolve/main/Vyesna-ZIT-lora_v1_000002750.safetensors?download=true", "/workspace/ComfyUI/models/loras", "Vyesna-ZIT-lora_v1_000002750.safetensors", "hf"),
        ("Vyesna ZIT v1 3000", "https://huggingface.co/rafatribe/Personnal_Loras/resolve/main/Vyesna-ZIT-lora_v1_000003000.safetensors?download=true", "/workspace/ComfyUI/models/loras", "Vyesna-ZIT-lora_v1_000003000.safetensors", "hf"),
        ("Vyesna ZIT v1 3250", "https://huggingface.co/rafatribe/Personnal_Loras/resolve/main/Vyesna-ZIT-lora_v1_000003250.safetensors?download=true", "/workspace/ComfyUI/models/loras", "Vyesna-ZIT-lora_v1_000003250.safetensors", "hf"),
    ],
    "🛠️ Upscale Models": [
        ("4x-UltraSharp", "https://huggingface.co/lokCX/4x-Ultrasharp/resolve/main/4x-UltraSharp.pth?download=true", "/workspace/ComfyUI/models/upscale_models", "4x-UltraSharp.pth", "hf"),
        ("RealESRGAN x4plus", "https://huggingface.co/lllyasviel/Annotators/resolve/main/RealESRGAN_x4plus.pth?download=true", "/workspace/ComfyUI/models/upscale_models", "RealESRGAN_x4plus.pth", "hf"),
        ("Remacri Original", "https://huggingface.co/jaideepsingh/upscale_models/resolve/main/remacri_original.pth?download=true", "/workspace/ComfyUI/models/upscale_models", "remacri_original.safetensors", "hf"),
    ]
}

# --- 3. INTERFACE GRAPHIQUE ---
style = {'description_width': 'initial'}
layout = widgets.Layout(width='98%')
checkboxes = {}
output_log = widgets.Output(layout={'border': '1px solid #444', 'height': '250px', 'overflow_y': 'scroll'})

accordion_children = []
titles = []

for category, items in db_models.items():
    vbox_items = []
    select_all = widgets.Button(description="Tout cocher", icon="check-square-o", button_style='info', layout=widgets.Layout(width='150px'))
    def on_select_all(b, cat=category):
        for k, val in checkboxes.items():
            if k.startswith(cat):
                val['cb'].value = True
    select_all.on_click(on_select_all)
    vbox_items.append(select_all)
    
    for item in items:
        name, url, path, filename, src = item
        key = f"{category}|{filename}"
        label = f"{name}"
        cb = widgets.Checkbox(value=False, description=label, style=style, layout=layout)
        checkboxes[key] = {'cb': cb, 'data': item}
        vbox_items.append(cb)
        
    accordion_children.append(widgets.VBox(vbox_items))
    titles.append(category)

accordion = widgets.Accordion(children=accordion_children)
for i, title in enumerate(titles):
    accordion.set_title(i, title)
    accordion.selected_index = 0

btn_download = widgets.Button(
    description='LANCER LE TÉLÉCHARGEMENT (Z-Image)',
    button_style='success', 
    layout=widgets.Layout(width='100%', height='60px', margin='20px 0px 0px 0px'),
    icon='download'
)

# --- 4. LOGIQUE DE TÉLÉCHARGEMENT ---
def run_download(b):
    output_log.clear_output()
    with output_log:
        print("🚀 Traitement des fichiers Z-Image...\n")
        count = 0
        for key, val in checkboxes.items():
            if val['cb'].value:
                count += 1
                item = val['data']
                name, url, path, filename, src = item
                
                os.makedirs(path, exist_ok=True)
                full_path = os.path.join(path, filename)
                
                if os.path.exists(full_path) and os.path.getsize(full_path) > 1024:
                    print(f"⏩ [SKIP] {name} est déjà là. (0 data utilisée)")
                    continue

                auth = ""
                final_url = url
                if src == "hf":
                    auth = f'--header="Authorization: Bearer {HF_TOKEN}"'
                elif src == "civitai":
                    sep = "&" if "?" in url else "?"
                    final_url = f"{url}{sep}token={CIVITAI_TOKEN}"
                
                print(f"⬇️ Téléchargement de : {name}")
                cmd = f'aria2c -x 16 -s 16 -k 1M -c {auth} -d "{path}" -o "{filename}" "{final_url}"'
                ret = os.system(cmd)
                if ret == 0: print(f"   ✅ OK.")
                else: print(f"   ⚠️ Erreur (Code {ret}).")
                    
        if count == 0: print("⚠️ Aucune case cochée.")
        else: print("\n🎉 Terminé.")

btn_download.on_click(run_download)

display(widgets.VBox([
    widgets.Label("🧬 INSTALLATION Z-IMAGE TURBO (Et LoRAs Persos)"),
    accordion,
    btn_download,
    output_log
]))

In [ ]:
# DÉMARRAGE COMFYUI (TOUT-EN-UN : FIX PYTORCH + DÉPENDANCES)

import os
import subprocess
import sys

# --- CONFIGURATION ---
COMFY_DIR = "/workspace/ComfyUI"
MANAGER_DIR = f"{COMFY_DIR}/custom_nodes/ComfyUI-Manager"

print("🚀 DÉMARRAGE COMFYUI (Mode Réparation & Mise à jour)...")

# 1. INSTALLATION OUTILS SYSTÈME
print("\n🔧 [1/6] Vérification outils système (ffmpeg, aria2)...")
subprocess.run(["apt-get", "update"], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
subprocess.run(["apt-get", "install", "-y", "ffmpeg", "aria2"], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

# 2. RÉPARATION PYTORCH (Le Fix Critique)
# On force la version 2.5.1 pour éviter l'erreur 'is_compiling'
print("\n🔥 [2/6] Mise à jour du Noyau PyTorch (Cela prend ~1 min)...")
# On vire xformers qui cause souvent des conflits avec les nouvelles versions
subprocess.run([sys.executable, "-m", "pip", "uninstall", "-y", "xformers"], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
# Installation de la version stable CUDA 12.4
subprocess.check_call([
    sys.executable, "-m", "pip", "install",
    "torch==2.5.1+cu124", "torchvision==0.20.1+cu124", "torchaudio==2.5.1+cu124",
    "--index-url", "https://download.pytorch.org/whl/cu124"
])

# 3. INSTALLATION DÉPENDANCES PYTHON
print("\n📦 [3/6] Installation des librairies manquantes...")
required_packages = [
    "numpy<2",           # Force NumPy 1.x (Crucial)
    "pydantic-settings", # Fix config
    "sentencepiece",     # Fix T5/Wan/Flux
    "aiohttp",           # Fix Manager
    "einops",            # Fix Core
    "av",                # Fix Vidéo
    "torchsde",          # Fix Samplers
    "scipy",             # Fix Math
    "insightface",       # Fix FaceSwap
    "onnxruntime-gpu",   # Fix FaceSwap
    "kornia",            # Fix Image Proc
    "spandrel",          # Fix Upscale
    "color-matcher"      # Fix Wan Color
]
subprocess.check_call([sys.executable, "-m", "pip", "install"] + required_packages)

# Requirements du Manager
if os.path.exists(f"{MANAGER_DIR}/requirements.txt"):
    print("   -> Dépendances ComfyUI-Manager...")
    subprocess.run([sys.executable, "-m", "pip", "install", "-r", f"{MANAGER_DIR}/requirements.txt"], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

# 4. TÉLÉCHARGEMENT MODÈLES INSIGHTFACE (AntelopeV2)
print("\n📥 [4/6] Vérification des modèles InsightFace...")
antelope_dir = f"{COMFY_DIR}/models/insightface/models/antelopev2"
os.makedirs(antelope_dir, exist_ok=True)
files_antelope = ["1k3d68.onnx", "2d106det.onnx", "genderage.onnx", "glintr100.onnx", "scrfd_10g_bnkps.onnx"]

for f in files_antelope:
    if not os.path.exists(f"{antelope_dir}/{f}"):
        print(f"   ⬇️ Téléchargement : {f}")
        subprocess.run(["aria2c", "-x", "16", "-s", "16", "-k", "1M", "-c", f"https://huggingface.co/okaris/antelopev2/resolve/main/{f}?download=true", "-d", antelope_dir, "-o", f], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

# 5. CONFIGURATION FINALE
print("\n⚙️ [5/6] Configuration finale...")
# On s'assure que le dossier custom_nodes est propre si besoin (optionnel)

# 6. LANCEMENT
print("\n🚀 [6/6] Lancement du serveur...")

run_script = """#!/bin/bash
cd /workspace/ComfyUI
# Arguments optimisés pour RunPod
python main.py --listen 0.0.0.0 --port 8188 --disable-auto-launch --cache-ram 8 --bf16-unet
"""

with open("run_comfy.sh", "w") as f:
    f.write(run_script)
os.chmod("run_comfy.sh", 0o755)

print("✅ ComfyUI démarre. (Si le terminal bloque ici, c'est bon signe : le serveur tourne !)")
!./run_comfy.sh

🚀 DÉMARRAGE COMFYUI (Mode Réparation & Mise à jour)...

🔧 [1/6] Vérification outils système (ffmpeg, aria2)...

🔥 [2/6] Mise à jour du Noyau PyTorch (Cela prend ~1 min)...
Looking in indexes: https://download.pytorch.org/whl/cu124
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.3/908.3 MB 10.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 115.7 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 117.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 138.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 145.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 169.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 13.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 25.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 37.2 


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python -m pip install --upgrade pip



📦 [3/6] Installation des librairies manquantes...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.1/75.1 kB 10.7 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of opencv-python-headless to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 92.1 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.9/51.9 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 98.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 147.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.4/64.4 kB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.7/39.7 MB 94.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 134


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python -m pip install --upgrade pip


   -> Dépendances ComfyUI-Manager...

📥 [4/6] Vérification des modèles InsightFace...

⚙️ [5/6] Configuration finale...

🚀 [6/6] Lancement du serveur...
✅ ComfyUI démarre. (Si le terminal bloque ici, c'est bon signe : le serveur tourne !)
[START] Security scan
[DONE] Security scan
## ComfyUI-Manager: installing dependencies done.
** ComfyUI startup time: 2025-12-13 08:37:37.021
** Platform: Linux
** Python version: 3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]
** Python executable: /usr/bin/python
** ComfyUI Path: /workspace/ComfyUI
** ComfyUI Base Folder Path: /workspace/ComfyUI
** User directory: /workspace/ComfyUI/user
** ComfyUI-Manager config path: /workspace/ComfyUI/user/__manager/config.ini
** Log path: /workspace/ComfyUI/user/comfyui.log

[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python -m pip install --upgrade pip

